<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import zipfile

zip_path = "/content/EdNet-KT3.zip"  # 로컬 경로

with zipfile.ZipFile(zip_path, 'r') as z:
    file_list = z.namelist()
    print("압축 안에 파일 개수:", len(file_list))
    print("앞 10개:", file_list[:10])

압축 안에 파일 개수: 297917
앞 10개: ['KT3/', '__MACOSX/._KT3', 'KT3/u533092.csv', 'KT3/u5901.csv', 'KT3/u734989.csv', 'KT3/u763609.csv', 'KT3/u347548.csv', 'KT3/u265925.csv', 'KT3/u319352.csv', 'KT3/u406983.csv']


In [26]:
import pandas as pd

with zipfile.ZipFile(zip_path, 'r') as z:
    csv_files = [f for f in z.namelist() if f.endswith(".csv")]
    print("CSV 파일 수:", len(csv_files))

    # 첫 번째 CSV만 예시로 열기
    with z.open(csv_files[0]) as f:
        df = pd.read_csv(f, low_memory=False)

print(df.iloc[:, :5].head())


CSV 파일 수: 297915
       timestamp action_type item_id   source user_answer
0  1552454260435       enter   e2575  my_note         NaN
1  1552454264737        quit   e2575  my_note         NaN
2  1552454266783       enter   e2482  my_note         NaN
3  1552454283275        quit   e2482  my_note         NaN
4  1552470265444       enter   e2482  my_note         NaN


In [32]:
import zipfile
import random
import pandas as pd

zip_path = "/content/EdNet-KT3.zip"

with zipfile.ZipFile(zip_path, 'r') as z:
    # 1. 유저별 CSV 파일 경로 가져오기
    user_files = [f for f in z.namelist() if f.startswith("KT3/") and f.endswith(".csv")]
    print("총 사용자 파일 수:", len(user_files))

    # 2. 500명 랜덤 추출 (총 수가 500명 미만이면 전부 추출)
    k = min(500, len(user_files))
    sample_files = random.sample(user_files, k)
    print("샘플 사용자 파일 예시:", sample_files[:5])

    # 3. 500명의 데이터를 하나의 DataFrame으로 합치기
    dfs = []
    for f in sample_files:
        with z.open(f) as fp:
            df = pd.read_csv(fp, low_memory=False)
            df["source_file"] = f.split("/")[-1].replace(".csv", "")  # 유저ID 기록
            dfs.append(df)

    df_sample = pd.concat(dfs, ignore_index=True)

print("샘플 데이터 크기:", df_sample.shape)
print(df_sample.head())


총 사용자 파일 수: 297915
샘플 사용자 파일 예시: ['KT3/u474769.csv', 'KT3/u633546.csv', 'KT3/u733730.csv', 'KT3/u547146.csv', 'KT3/u333479.csv']
샘플 데이터 크기: (181428, 7)
       timestamp action_type item_id     source user_answer platform  \
0  1549275419383       enter   b4790  diagnosis         NaN   mobile   
1  1549275434826     respond   q6258  diagnosis           b   mobile   
2  1549275439430      submit   b4790  diagnosis         NaN   mobile   
3  1549275439545       enter   b4512  diagnosis         NaN   mobile   
4  1549275454532     respond   q5980  diagnosis           c   mobile   

  source_file  
0     u474769  
1     u474769  
2     u474769  
3     u474769  
4     u474769  


In [19]:
!ls -lh /content/drive/MyDrive/EdNet-KT3.zip
!file /content/drive/MyDrive/EdNet-KT3.zip

-rw------- 1 root root 543M Sep 22 04:09 /content/drive/MyDrive/EdNet-KT3.zip
/content/drive/MyDrive/EdNet-KT3.zip: Zip archive data, at least v2.0 to extract, compression method=store
